# Sentiment Analysis of ChatGPT Conversations in GitHub Pull Requests and Issues

*How does the sentiment of ChatGPT prompts and responses in GitHub pull requests and issues vary across different types of requests?*

**Hypothesis**: I hypothesize that PRs would have higher sentiment because it involves finalizing work and collaboration, which seems more positive. Issues seem more focused on identifying problems and bugs, which I would think involves frustration. PRs feels more like progress, while Issues seems more about setbacks.


### Section 1: Loading and Normalizing the Datasets

In [2]:
import pandas as pd
import json
from textblob import TextBlob
import altair as alt

# Load the PR dataset and Issues dataset, then normalizing them, and then concatenating them
df_pr = pd.read_json("20230727_195927_pr_sharings.json")
df_issues = pd.read_json("20230727_195941_issue_sharings.json")

df_pr = pd.json_normalize(df_pr['Sources'])
df_issues = pd.json_normalize(df_issues['Sources'])

df_pr['SourceType'] = 'Pull Request'
df_issues['SourceType'] = 'Issue'
df_combined = pd.concat([df_pr, df_issues], ignore_index=True)

#### Loading and Normalizing the PR Dataset

In this section, we load the pull request dataset and normalize the Sources column. We also add a SourceType column to indicate that the rows come from PRs.

### Section 2: Function to Extract Prompts

In [3]:
def extract_first_last_prompts(chatgpt_sharing):

    # This function exracts the first and last prompots from chatGPT Conversations
    try:
        if isinstance(chatgpt_sharing, str):
            chatgpt_sharing = json.loads(chatgpt_sharing)

        if isinstance(chatgpt_sharing, list) and len(chatgpt_sharing) > 0:
            chatgpt_sharing = chatgpt_sharing[0]

        conversations = chatgpt_sharing.get('Conversations', [])
        prompts = [convo.get('Prompt', '').strip() for convo in conversations if convo.get('Prompt')]

        first_prompt = prompts[0] if prompts else None
        last_prompt = prompts[-1] if prompts else None

        return first_prompt, last_prompt
    except Exception as e:
        return None, None

#### Extracting Prompts from ChatGPT Conversations

This function extracts the first and last prompts from the ChatgptSharing field in each row of the dataset. We handle JSON parsing and ensure we only grab the prompts from the conversation.

### Section 3: Sentiment Analysis Function

In [4]:
def analyze_sentiment(text):

    # I use this function to get sentiment using Textblob
    #Sentiments vary from -1 to 1

    if not isinstance(text, str) or text.strip() == "":
        return None  
    
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  



#### Sentiment Analysis Function

This function uses the TextBlob library to perform sentiment analysis on the extracted prompts. It returns a polarity score, ranging from -1 (negative sentiment) to 1 (positive sentiment).

### Section 4: Processing the PR Data

In [5]:
sentiment_first = []
sentiment_last = []

for index, row in df_combined.iterrows():
    first_prompt, last_prompt = extract_first_last_prompts(row['ChatgptSharing'])
    
    sentiment_first.append(analyze_sentiment(first_prompt))
    sentiment_last.append(analyze_sentiment(last_prompt))

df_combined['SentimentScore_FirstPrompt'] = sentiment_first
df_combined['SentimentScore_LastPrompt'] = sentiment_last

print(df_combined[['SentimentScore_FirstPrompt', 'SentimentScore_LastPrompt']].head())

   SentimentScore_FirstPrompt  SentimentScore_LastPrompt
0                    0.172917                   0.118182
1                    0.375000                   0.268182
2                   -0.012603                   0.000000
3                   -0.079167                  -0.079167
4                    0.000000                   0.000000


### Processing the PR Data

In this section, we loop through the PR dataset, extract the first and last prompts from the ChatgptSharing column, and calculate sentiment scores. These scores are then added to the dataset as new columns.

### Section 5: Plotting Data for First and Last Prompts

In [18]:
import altair as alt

#Plots created with ChatGPT

# Create a DataFrame from your sentiment scores
df_combined = pd.DataFrame({
    'SentimentScore_FirstPrompt': sentiment_first,
    'SentimentScore_LastPrompt': sentiment_last,
    'SourceType': df_combined['SourceType'],
    'Index': df_combined.index
})

# Create an Altair plot
alt_chart = alt.Chart(df_combined).transform_fold(
    ['SentimentScore_FirstPrompt', 'SentimentScore_LastPrompt'],
    as_=['PromptType', 'SentimentScore']
).mark_point(filled=True).encode(
    alt.X('SentimentScore:Q', title='Sentiment Score'),
    alt.Y('PromptType:N', title='Prompt Type'),
    color='SourceType:N',  # Color by SourceType (PR vs Issues)
    tooltip=['PromptType:N', 'SentimentScore:Q', 'SourceType:N']
).properties(
    title='Sentiment Score Distribution for First and Last Prompts'
)

alt_chart.show()

alt.Chart(...)

In [19]:
import altair as alt
import pandas as pd

# Assuming df_combined contains the data with sentiment scores
# Reshape the dataframe to make it easier to plot
df_melted = df_combined.melt(id_vars=["SourceType"], value_vars=["SentimentScore_FirstPrompt", "SentimentScore_LastPrompt"], 
                             var_name="PromptType", value_name="SentimentScore")

# Filter the data for Issues and PRs separately
df_issues = df_melted[df_melted['SourceType'] == 'Issue']
df_prs = df_melted[df_melted['SourceType'] == 'Pull Request']

# Create a chart for Issues with proper title
issues_chart = alt.Chart(df_issues).mark_bar().encode(
    alt.X('PromptType:N', title='Prompt Type', axis=alt.Axis(labels=True, titleFontSize=14, labelFontSize=12)),
    alt.Y('SentimentScore:Q', title='Sentiment Score', axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    color='PromptType:N',  # Different colors for first and last prompt
    tooltip=['SourceType:N', 'SentimentScore:Q', 'PromptType:N']
).properties(
    title='Issues: Sentiment Score - First vs Last Prompt',
    width=300
)

# Create a chart for Pull Requests with proper title
prs_chart = alt.Chart(df_prs).mark_bar().encode(
    alt.X('PromptType:N', title='Prompt Type', axis=alt.Axis(labels=True, titleFontSize=14, labelFontSize=12)),
    alt.Y('SentimentScore:Q', title='Sentiment Score', axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    color='PromptType:N',  # Different colors for first and last prompt
    tooltip=['SourceType:N', 'SentimentScore:Q', 'PromptType:N']
).properties(
    title='Pull Requests - Sentiment Score Score: First vs Last Prompt for Pull Requests',
    width=300
)

# Combine the charts side by side using a horizontal layout
combined_chart = alt.hconcat(issues_chart, prs_chart)

# Display the final chart
combined_chart.show()

alt.HConcatChart(...)

### Section 6: Expanding Data for all Prompts

In [8]:
df_pr = pd.read_json("20230727_195927_pr_sharings.json")
df_issues = pd.read_json("20230727_195941_issue_sharings.json")
df_pr = pd.json_normalize(df_pr['Sources'])
df_issues = pd.json_normalize(df_issues['Sources'])
df_pr['SourceType'] = 'Pull Request'
df_issues['SourceType'] = 'Issue'
df_combined = pd.concat([df_pr, df_issues], ignore_index=True)

def extract_prompts_and_calculate_sentiment(chatgpt_sharing):
    try:
        if isinstance(chatgpt_sharing, str):
            chatgpt_sharing = json.loads(chatgpt_sharing)

        if isinstance(chatgpt_sharing, list) and len(chatgpt_sharing) > 0:
            chatgpt_sharing = chatgpt_sharing[0]

        # Extract conversations (all prompts)
        conversations = chatgpt_sharing.get('Conversations', [])
        prompts = [convo.get('Prompt', '').strip() for convo in conversations if convo.get('Prompt')]

        # Calculate sentiment for each prompt
        sentiment_scores = [TextBlob(prompt).sentiment.polarity for prompt in prompts]
        return sentiment_scores
    except Exception as e:
        return []

# Calculate sentiment for each row (prompts will be processed and averaged)
df_combined['All_Prompt_Sentiment_Scores'] = df_combined['ChatgptSharing'].apply(extract_prompts_and_calculate_sentiment)

# Calculate the average sentiment score for each row
df_combined['Average_Sentiment'] = df_combined['All_Prompt_Sentiment_Scores'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else None)

# Display the updated dataframe
print(df_combined[['SourceType', 'Average_Sentiment']].head())

# Now let's plot the averages
df_averages = df_combined.groupby('SourceType')['Average_Sentiment'].mean().reset_index()

# Create a bar plot of the average sentiment for Issues and PRs
chart = alt.Chart(df_averages).mark_bar().encode(
    alt.X('SourceType:N', title='Source Type'),
    alt.Y('Average_Sentiment:Q', title='Average Sentiment', axis=alt.Axis(titleFontSize=14, labelFontSize=12)),
    color='SourceType:N',  # Different colors for Issues and PRs
    tooltip=['SourceType:N', 'Average_Sentiment:Q']
).properties(
    title='Average Sentiment for Issues and Pull Requests',
    width=300
)

# Display the chart
chart.show()

     SourceType  Average_Sentiment
0  Pull Request          -0.008691
1  Pull Request           0.053488
2  Pull Request           0.120664
3  Pull Request          -0.079167
4  Pull Request           0.000000


alt.Chart(...)

### Section 7: Analysis

So from the chart above we insteard see the average sentiment scores for Issues and Pull Requests this time based on all ChatGPT prompts.

It shows a signifncant change between issues with more positive sentiment on average on Issues than on Pull Requests.

We hypothesize that this might be because Issues is often on identfiying problems making new changes. While contrasting this Pull Request smight have more technical discussion and feedback, which has a lower sentiment score. Maybe the developer also critisizes the code or feedback hes receiving from ChatGPT. 

This was acutally the opposite of the original hypothesis I had for this section.

Overall, the difference in sentiment could give insights into how developers use ChatGPT in different contexts, like debugging versus finalizing changes.